# Day 51 - Introduction to Convolutional Neural Networks (CNNs)

Welcome to Day 51 of the 100 Days of Machine Learning! Today, we begin our journey into **Convolutional Neural Networks (CNNs)**, one of the most important architectures in deep learning, particularly for computer vision tasks.

## Introduction

Convolutional Neural Networks have revolutionized the field of computer vision. From image classification to object detection, face recognition to medical image analysis, CNNs have achieved remarkable success.

### Why CNNs?

Traditional fully connected neural networks face several challenges when working with images:
- **High dimensionality**: A 224×224 RGB image has 150,528 input features
- **Loss of spatial structure**: Flattening destroys spatial relationships between pixels
- **No translation invariance**: The network can't recognize the same object in different positions
- **Too many parameters**: Leading to overfitting and computational inefficiency

CNNs address these issues through:
1. **Local connectivity**: Each neuron connects only to a small region
2. **Parameter sharing**: Same weights used across the entire image
3. **Translation equivariance**: Features detected regardless of position

### Learning Objectives

By the end of this lesson, you will:
- Understand the core concepts of convolutional neural networks
- Learn how convolution operations extract features from images
- Implement basic CNNs using TensorFlow/Keras
- Visualize feature maps and understand what CNNs learn
- Apply CNNs to image classification tasks

## Theory: CNN Architecture

### The Building Blocks

A typical CNN consists of three main types of layers:

1. **Convolutional Layers**: Extract features using learnable filters
2. **Pooling Layers**: Reduce spatial dimensions while retaining important information
3. **Fully Connected Layers**: Perform final classification based on extracted features

### The Convolution Operation

The fundamental operation in CNNs is the **convolution**. Given an input image $I$ and a kernel (filter) $K$, the convolution operation is defined as:

$$S(i,j) = (I * K)(i,j) = \sum_m \sum_n I(i+m, j+n) \cdot K(m,n)$$

Where:
- $S$ is the output feature map
- $I$ is the input image
- $K$ is the kernel/filter
- $(i,j)$ is the position in the output
- $(m,n)$ iterates over the kernel dimensions

### Key Parameters

**1. Kernel Size**: The dimensions of the filter (e.g., 3×3, 5×5)

**2. Stride**: How much the kernel moves at each step. With stride $s$, the output dimension is:

$$O = \left\lfloor \frac{W - K + 2P}{s} \right\rfloor + 1$$

Where $W$ is input width, $K$ is kernel size, and $P$ is padding.

**3. Padding**: Adding zeros around the input to control output size
- **Valid padding**: No padding, output shrinks
- **Same padding**: Padding such that output size equals input size (when stride=1)

**4. Number of Filters**: Each filter learns to detect a different feature

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

## Manual Convolution: Understanding the Basics

Let's implement a simple 2D convolution from scratch to understand how it works.

In [2]:
def convolve2d(image, kernel):
    """Perform 2D convolution manually"""
    image_h, image_w = image.shape
    kernel_h, kernel_w = kernel.shape
    
    # Calculate output dimensions
    output_h = image_h - kernel_h + 1
    output_w = image_w - kernel_w + 1
    
    # Initialize output
    output = np.zeros((output_h, output_w))
    
    # Perform convolution
    for i in range(output_h):
        for j in range(output_w):
            # Extract region
            region = image[i:i+kernel_h, j:j+kernel_w]
            # Element-wise multiplication and sum
            output[i, j] = np.sum(region * kernel)
    
    return output

# Create a simple test image
test_image = np.array([
    [1, 2, 3, 4, 5, 6],
    [7, 8, 9, 10, 11, 12],
    [13, 14, 15, 16, 17, 18],
    [19, 20, 21, 22, 23, 24],
    [25, 26, 27, 28, 29, 30],
    [31, 32, 33, 34, 35, 36]
])

# Define a simple averaging kernel
avg_kernel = np.ones((3, 3)) / 9

# Apply convolution
output = convolve2d(test_image, avg_kernel)

print(f"Input Image Shape: {test_image.shape}")
print(f"Kernel Shape: {avg_kernel.shape}")
print(f"Output Feature Map Shape: {output.shape}")

Input Image Shape: (6, 6)
Kernel Shape: (3, 3)
Output Feature Map Shape: (4, 4)


## Feature Detection: Edge Detection Kernels

Different kernels detect different features. Let's explore some classic edge detection kernels.

In [3]:
# Create a simple test image with edges
simple_image = np.zeros((10, 10))
simple_image[:, 5:] = 1  # Vertical edge
simple_image[5:, :] += 1  # Horizontal edge

# Define edge detection kernels
vertical_edge = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])  # Sobel vertical

horizontal_edge = np.array([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
])  # Sobel horizontal

# Apply edge detection
vertical_features = signal.convolve2d(simple_image, vertical_edge, mode='valid')
horizontal_features = signal.convolve2d(simple_image, horizontal_edge, mode='valid')

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(simple_image, cmap='gray')
axes[0].set_title('Original Image', fontsize=12)
axes[0].axis('off')

axes[1].imshow(vertical_features, cmap='gray')
axes[1].set_title('Vertical Edge Detection', fontsize=12)
axes[1].axis('off')

axes[2].imshow(horizontal_features, cmap='gray')
axes[2].set_title('Horizontal Edge Detection', fontsize=12)
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Pooling Layers

**Pooling layers** reduce the spatial dimensions of feature maps while retaining important information.

### Max Pooling

Takes the maximum value from each pool region:

$$P(i,j) = \max_{(m,n) \in R_{ij}} A(m,n)$$

Where $R_{ij}$ is the pooling region and $A$ is the activation map.

### Benefits:
- Reduces computational cost
- Provides translation invariance
- Helps prevent overfitting
- Increases receptive field

In [4]:
def max_pooling(feature_map, pool_size=2):
    """Apply max pooling to a feature map"""
    h, w = feature_map.shape
    out_h, out_w = h // pool_size, w // pool_size
    
    pooled = np.zeros((out_h, out_w))
    
    for i in range(out_h):
        for j in range(out_w):
            region = feature_map[
                i*pool_size:(i+1)*pool_size,
                j*pool_size:(j+1)*pool_size
            ]
            pooled[i, j] = np.max(region)
    
    return pooled

# Test max pooling
test_feature = np.array([
    [1, 3, 2, 4],
    [5, 6, 1, 2],
    [7, 2, 8, 3],
    [1, 4, 6, 9]
])

pooled = max_pooling(test_feature, pool_size=2)

print(f"Original shape: {test_feature.shape}")
print(f"After max pooling (2x2): {pooled.shape}")
print(f"\nOriginal:\n{test_feature}")
print(f"\nAfter Max Pooling:\n{pooled}")

Original shape: (4, 4)
After max pooling (2x2): (2, 2)
\nOriginal:\n[[1 3 2 4]\n [5 6 1 2]\n [7 2 8 3]\n [1 4 6 9]]\n\nAfter Max Pooling:\n[[6. 4.]\n [7. 9.]]


## Building Your First CNN with TensorFlow/Keras

Now let's build a complete CNN for image classification using the MNIST dataset.

In [5]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape to add channel dimension (grayscale = 1 channel)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Convert labels to categorical
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Number of classes: {len(np.unique(y_train))}")

Training set: (60000, 28, 28, 1)
Test set: (10000, 28, 28, 1)
Number of classes: 10


In [6]:
# Visualize some training examples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(X_train[i].reshape(28, 28), cmap='gray')
    axes[i].set_title(f'Label: {y_train[i]}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('Sample MNIST Digits', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [7]:
# Build the CNN model
model = keras.Sequential([
    # First convolutional block
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Second convolutional block
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten and fully connected layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [8]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train_cat,
    batch_size=128,
    epochs=5,
    validation_data=(X_test, y_test_cat),
    verbose=1
)

Epoch 1/5
469/469 [==============================] - 45s 95ms/step - loss: 0.2145 - accuracy: 0.9356 - val_loss: 0.0545 - val_accuracy: 0.9827
Epoch 2/5
469/469 [==============================] - 43s 92ms/step - loss: 0.0783 - accuracy: 0.9762 - val_loss: 0.0392 - val_accuracy: 0.9871
Epoch 3/5
469/469 [==============================] - 44s 93ms/step - loss: 0.0592 - accuracy: 0.9817 - val_loss: 0.0338 - val_accuracy: 0.9885
Epoch 4/5
469/469 [==============================] - 43s 92ms/step - loss: 0.0489 - accuracy: 0.9849 - val_loss: 0.0304 - val_accuracy: 0.9895
Epoch 5/5
469/469 [==============================] - 44s 93ms/step - loss: 0.0421 - accuracy: 0.9866 - val_loss: 0.0287 - val_accuracy: 0.9904


In [9]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test_cat)

print(f"\nTest Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

313/313 [==============================] - 3s 10ms/step - loss: 0.0287 - accuracy: 0.9904
\nTest Accuracy: 99.04%
Test Loss: 0.0287


In [10]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', marker='o')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Model Accuracy Over Epochs', fontsize=14)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', marker='o')
axes[1].plot(history.history['val_loss'], label='Validation Loss', marker='s')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Model Loss Over Epochs', fontsize=14)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Visualizing What CNNs Learn

Let's visualize the feature maps (activations) from the convolutional layers to understand what the CNN is learning.

In [11]:
# Create a model that outputs intermediate activations
layer_outputs = [layer.output for layer in model.layers[:4]]  # First 4 layers
activation_model = keras.Model(inputs=model.input, outputs=layer_outputs)

# Get activations for a test image
test_image = X_test[0:1]  # First test image
activations = activation_model.predict(test_image, verbose=0)

# Visualize first convolutional layer activations
first_layer_activation = activations[0]
print(f"First conv layer output shape: {first_layer_activation.shape}")

# Plot the original image and feature maps
fig = plt.figure(figsize=(16, 8))

# Original image
plt.subplot(4, 9, 1)
plt.imshow(test_image[0, :, :, 0], cmap='gray')
plt.title('Original', fontsize=9)
plt.axis('off')

# Plot 32 feature maps from first conv layer
for i in range(32):
    plt.subplot(4, 9, i + 2)
    plt.imshow(first_layer_activation[0, :, :, i], cmap='viridis')
    plt.title(f'F{i+1}', fontsize=8)
    plt.axis('off')

plt.suptitle('First Convolutional Layer Feature Maps (32 filters)', fontsize=14)
plt.tight_layout()
plt.show()

First conv layer output shape: (1, 26, 26, 32)


In [12]:
# Make predictions on test samples
predictions = model.predict(X_test[:10], verbose=0)
predicted_classes = np.argmax(predictions, axis=1)

# Visualize predictions
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(X_test[i].reshape(28, 28), cmap='gray')
    
    # Color code: green for correct, red for incorrect
    color = 'green' if predicted_classes[i] == y_test[i] else 'red'
    axes[i].set_title(
        f'True: {y_test[i]}\nPred: {predicted_classes[i]} ({predictions[i][predicted_classes[i]]*100:.1f}%)',
        fontsize=10,
        color=color
    )
    axes[i].axis('off')

plt.suptitle('CNN Predictions on Test Set', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Hands-On Exercise

### Exercise 1: Experiment with Architecture

Modify the CNN architecture and observe how it affects performance:

```python
# Try these modifications:
# 1. Add a third convolutional layer
# 2. Change the number of filters (e.g., 16, 32, 128)
# 3. Try different kernel sizes (5x5, 7x7)
# 4. Experiment with different activation functions
# 5. Add batch normalization layers

model_modified = keras.Sequential([
    layers.Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Add your third layer here
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Compile and train your modified model
```

### Exercise 2: Implement Different Pooling

Replace MaxPooling with AveragePooling and compare results:

```python
# Replace:
layers.MaxPooling2D(pool_size=(2, 2))
# With:
layers.AveragePooling2D(pool_size=(2, 2))
```

### Exercise 3: Data Augmentation

Add data augmentation to improve model robustness:

```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

# Train with augmented data
# history = model.fit(datagen.flow(X_train, y_train_cat, batch_size=128), ...)
```

### Questions to Explore:

1. How does adding more convolutional layers affect accuracy and training time?
2. What happens when you use very small (1x1) or very large (7x7) kernels?
3. How does the number of filters in each layer impact performance?
4. Can you achieve >99.5% accuracy on the test set?

## Understanding Parameter Count

Let's calculate the number of parameters in our CNN manually.

### First Conv Layer (Conv2D: 32 filters, 3×3 kernel)

$$\text{Params} = (K_h \times K_w \times C_{in} + 1) \times C_{out}$$
$$= (3 \times 3 \times 1 + 1) \times 32 = 10 \times 32 = 320$$

Where:
- $K_h, K_w$ = kernel height and width (3, 3)
- $C_{in}$ = input channels (1 for grayscale)
- $C_{out}$ = output channels (32 filters)
- +1 for bias term

### Second Conv Layer (Conv2D: 64 filters, 3×3 kernel, 32 input channels)

$$\text{Params} = (3 \times 3 \times 32 + 1) \times 64 = 289 \times 64 = 18,496$$

### Fully Connected Layer (Dense: 1600 → 128)

$$\text{Params} = (1600 + 1) \times 128 = 204,928$$

### Output Layer (Dense: 128 → 10)

$$\text{Params} = (128 + 1) \times 10 = 1,290$$

### Total Parameters

$$\text{Total} = 320 + 18,496 + 204,928 + 1,290 = 225,034$$

This matches the model summary!

## Receptive Field

The **receptive field** is the region of the input image that affects a particular feature in the output.

For our CNN:

1. **After Conv1 (3×3)**: Each neuron sees a 3×3 region
2. **After MaxPool1 (2×2, stride 2)**: Receptive field becomes 4×4
3. **After Conv2 (3×3)**: Receptive field grows to 10×10
4. **After MaxPool2 (2×2, stride 2)**: Receptive field becomes 12×12

### Formula for Receptive Field:

$$r_{out} = r_{in} + (k - 1) \times \prod_{i=1}^{n} s_i$$

Where:
- $r_{out}$ = output receptive field
- $r_{in}$ = input receptive field
- $k$ = kernel size
- $s_i$ = stride of previous layers

Deeper networks with more layers can capture larger spatial context!

## Key Takeaways

### 1. Core Concepts
- **CNNs use convolution operations** to extract hierarchical features from images
- **Local connectivity** and **parameter sharing** make CNNs efficient for image data
- **Pooling layers** reduce spatial dimensions and provide translation invariance

### 2. CNN Architecture
- **Convolutional layers** detect features using learnable filters
- **Activation functions** (ReLU) introduce non-linearity
- **Pooling layers** downsample feature maps
- **Fully connected layers** perform final classification

### 3. Key Parameters
- **Kernel size**: Determines the local region size (typically 3×3 or 5×5)
- **Number of filters**: Controls feature diversity and model capacity
- **Stride**: Controls how much the filter moves (usually 1 for conv, 2 for pooling)
- **Padding**: Controls output spatial dimensions

### 4. Important Principles
- **Early layers** detect low-level features (edges, corners)
- **Deeper layers** detect high-level features (shapes, objects)
- **Receptive field** grows with network depth
- **Parameter efficiency**: CNNs have far fewer parameters than fully connected networks

### 5. Best Practices
- Start with **simple architectures** and gradually increase complexity
- Use **data augmentation** to improve generalization
- Apply **dropout** to prevent overfitting
- Monitor **both training and validation metrics**
- **Visualize feature maps** to understand what the network learns

### Mathematical Insights

**Convolution operation**:
$$S(i,j) = \sum_m \sum_n I(i+m, j+n) \cdot K(m,n)$$

**Output size calculation**:
$$O = \left\lfloor \frac{W - K + 2P}{S} \right\rfloor + 1$$

**Parameter count for conv layer**:
$$\text{Params} = (K_h \times K_w \times C_{in} + 1) \times C_{out}$$

## Further Resources

### Papers
1. **LeCun et al. (1998)** - "Gradient-Based Learning Applied to Document Recognition" (LeNet-5)
   - The foundational CNN paper introducing LeNet

2. **Krizhevsky et al. (2012)** - "ImageNet Classification with Deep Convolutional Neural Networks" (AlexNet)
   - Breakthrough paper that sparked the deep learning revolution

3. **Zeiler & Fergus (2014)** - "Visualizing and Understanding Convolutional Networks"
   - Excellent resource for understanding what CNNs learn

### Online Resources
- **CS231n: Convolutional Neural Networks for Visual Recognition** (Stanford)
  - http://cs231n.stanford.edu/
  - Comprehensive course on CNNs and computer vision

- **Deep Learning Book** by Goodfellow, Bengio, and Courville
  - Chapter 9: Convolutional Networks
  - https://www.deeplearningbook.org/

- **TensorFlow CNN Tutorial**
  - https://www.tensorflow.org/tutorials/images/cnn

### Interactive Tools
- **CNN Explainer** - Interactive visualization of CNNs
  - https://poloclub.github.io/cnn-explainer/

- **ConvNetJS** - Demo of CNNs running in the browser
  - https://cs.stanford.edu/people/karpathy/convnetjs/

### Next Steps
- **Lesson 52**: CNN Building Blocks - Deep dive into convolution and pooling
- **Lesson 53**: Building and Training CNN Architectures
- **Lesson 54**: Advanced CNN Architectures (AlexNet, VGG, ResNet)
- **Lesson 55**: Transfer Learning and Fine-Tuning

### Practice Datasets
- **CIFAR-10/100**: Color images of objects (10 or 100 classes)
- **Fashion-MNIST**: Grayscale images of clothing items
- **ImageNet**: Large-scale image classification dataset
- **COCO**: Complex object detection and segmentation

---

**Congratulations!** You've completed Day 51 and learned the fundamentals of Convolutional Neural Networks. Tomorrow, we'll dive deeper into CNN building blocks and explore more advanced architectures.